# False clouds in ABI static features

Clouds that do not move or evolve are more likely to be false clouds. This program overlays ABI imagery in order to see which features do not evolve with time. 

ABI data is from /mnt/grb/goes16/ on smiller2

In [ ]:
import xarray as xr
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from sklearn.preprocessing import normalize
import fnmatch
import pickle
import timeit

In [ ]:
#--- Filter to the region

#---Georges Bank
min_lon = -71.5
min_lat = 37
max_lon = -64
max_lat = 42.5

## Visualizing a specific time

In [ ]:
date = '20230909'

nlct_path = "/home/jturner/false_low_clouds_2023/Nighttime_Low_Cloud_Test/nlct_data/"+date+"/"

filename = 'goes_e_ntlc_'+date[0:4]+'_'+date[4:6]+'_'+date[6:8]+'_00H_00M'

In [ ]:
file = open(nlct_path+filename, 'rb')
BTD = pickle.load(file)
file.close()

Filter the NLCT to the region of interest:

In [ ]:
BTD_c = BTD.where((BTD.lat>min_lat) & (BTD.lat<max_lat) & (BTD.lon>min_lon) & (BTD.lon<max_lon))
BTD_c = BTD_c.dropna(dim="y", how="all")
BTD_c = BTD_c.dropna(dim="x", how="all")

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.YlGnBu
#levels = np.linspace(0, np.amax(BTD), 31)

c=ax.contourf(BTD_c.lon, BTD_c.lat, BTD_c[0], cmap=cmap) #levels=levels)
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('ABI BTD (11-3.9): ' + filename[12:])
clb.set_label('BTD')

ax.add_feature(cfeature.STATES)

## Combining the range of times

In [ ]:
#--- Initialize an array of 1's in the shape of the final image
BTD_prod = [ [1]*len(BTD_c[0][0]) for i in range(len(BTD_c[0]))]

#--- Starting the timer
starttime = timeit.default_timer()

#--- Include all times or just top of hour (TOH)
all_files = sorted(os.listdir(nlct_path))
toh_files = [filename for filename in all_files if filename.endswith('00M')]

for filename in toh_files:
        print(filename)
        
        #--- Open the NLCT file
        file = open(nlct_path+filename, 'rb')
        BTD = pickle.load(file)
        file.close()
        
        #--- Crop to the specified region
        BTD = BTD.where((BTD.lat>min_lat) & (BTD.lat<max_lat) & (BTD.lon>min_lon) & (BTD.lon<max_lon))
        BTD = BTD.dropna(dim="y", how="all")
        BTD = BTD.dropna(dim="x", how="all")
        
        #--- Set negative values to zero
        BTD = np.where(BTD < 0, 0, BTD)
        
        #--- V1: Multiply to only enhance static clouds
        #BTD_prod = BTD_prod*BTD
        
        #--- V1: Limit runaway large values to 3
        #BTD_prod = np.where(BTD_prod > 3, 3, BTD_prod)
        
        #--- V2: Sum to enhance static clouds
        BTD_prod = BTD_prod+BTD
        
        
print("Completed: ", timeit.default_timer() - starttime)

V2 processing of summed BTD (do not run for V1):

In [ ]:
#--- V2: Limit to top 25%
BTD_prod = np.nan_to_num(BTD_prod, nan=0)
BTD_prod = BTD_prod/np.max(BTD_prod)
limit = 0.75
BTD_prod = np.where(BTD_prod < limit, 0, BTD_prod)
BTD_prod = np.where(BTD_prod > limit, 1, BTD_prod)

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.YlGnBu
#levels = np.linspace(0, 3, 31)
#levels = np.linspace(0, 1, 11)

c=ax.contourf(BTD_c.lon, BTD_c.lat, BTD_prod[0], cmap=cmap)#, levels=levels, extend='neither')
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('Static Features V2: '+date, fontsize=30)
clb.set_label('BTD')

ax.add_feature(cfeature.STATES)

In [ ]:
static_value = np.nansum(BTD_prod)
static_value